In [3]:
import datetime
from geneformer import Classifier
from pathlib import Path

current_date = datetime.datetime.now()
datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}{current_date.hour:02d}{current_date.minute:02d}{current_date.second:02d}"
datestamp_min = (
    f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}"
)

output_prefix = "3_ind_classify"
output_dir = f"./{output_prefix}/{datestamp}"
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [4]:
training_args = {
    "num_train_epochs": 0.9,
    "learning_rate": 0.000804,
    "lr_scheduler_type": "polynomial",
    "warmup_steps": 1812,
    "weight_decay": 0.258828,
    "per_device_train_batch_size": 12,
    "seed": 73,
    "epochs": 10, 
    
}
cc = Classifier(
    classifier="cell",
    cell_state_dict={"state_key": "incident_cad", "states": "all"},
    filter_data=None,
    training_args=training_args,
    max_ncells=None,
    freeze_layers=2,
    num_crossval_splits=1,
    forward_batch_size=4,
    nproc=16,
)

In [5]:
# previously balanced splits with prepare_data and validate functions
# argument attr_to_split set to "individual" and attr_to_balance set to ["disease","lvef","age","sex","length"]
train_ids = list(range(128))
eval_ids = [21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
test_ids = [31, 32, 33, 34, 35, 36, 37, 38, 39, 40]

train_test_id_split_dict = {
    "attr_key": "eid",
    "train": train_ids + eval_ids,
    "test": test_ids,
}


cc.prepare_data(
    input_data_file="/home/xutingfeng/ukb/project/ppp_prediction/2_Data/GeneFormer/imputed/train",
    output_directory=output_dir,
    output_prefix=output_prefix,
    split_id_dict=train_test_id_split_dict,
    test_size=0.2,
)

Saving the dataset (0/1 shards):   0%|          | 0/128 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

In [6]:
# 6 layer Geneformer: https://huggingface.co/ctheodoris/Geneformer/blob/main/model.safetensors

model_used = "6_layer"
if model_used == "6_layer":
    model_directory = "/home/xutingfeng/github_code/others/Geneformer/"
elif model_used == "12_layer":
    model_directory = (
        "/home/xutingfeng/github_code/others/Geneformer/geneformer-12L-30M"
    )


all_metrics = cc.validate(
    model_directory=model_directory,  # 12L
    prepared_input_data_file=f"{output_dir}/{output_prefix}_labeled_train.dataset",
    id_class_dict_file=f"{output_dir}/{output_prefix}_id_class_dict.pkl",
    output_directory=output_dir,
    output_prefix=output_prefix,
    split_id_dict=None,
)

  0%|          | 0/1 [00:00<?, ?it/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/xutingfeng/github_code/others/Geneformer/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


****** Validation split: 1/1 ******



/home/xutingfeng/miniforge3/envs/work/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
/home/xutingfeng/github_code/others/Geneformer/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.629800,0.656620,0.884615,0.668085


  0%|          | 0/7 [00:00<?, ?it/s]

/home/xutingfeng/github_code/others/Geneformer/geneformer/evaluation_utils.py:190: SyntaxWarning: invalid escape sequence '\p'
  label = f"{model_name} (AUC {roc_auc:0.2f} $\pm$ {roc_auc_sd:0.2f})"


RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

In [2]:
import torch

torch.cuda.is_available()

True

In [6]:
import pandas as pd

pd.read_pickle(
    "/home/xutingfeng/ukb/project/ppp_prediction/3_ind_classify/240324151736/3_ind_classify_id_class_dict.pkl"
)

{0: 0.0, 1: 1.0}